In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import os
import sys
sys.path.append('/kaggle/input/visualizing')
from skimage import transform
from torchvision import models
from torch.utils.data.sampler import Sampler
from torch.utils.data import DataLoader
import sklearn.metrics as metrics
from torch.utils.data import Dataset, SubsetRandomSampler
from torchvision import transforms
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import skopt
from skopt import gp_minimize, dummy_minimize
from skopt.space import Real, Categorical, Integer
from skopt.utils import use_named_args
from skopt.plots import plot_convergence, plot_objective

device = torch.device("cuda:0")
print(f"Using device: {device}")

class HotdogRecognitionDataset(Dataset):
    def __init__(self, folder, transform=None):
        self.folder = folder
        self.transform = transform
        self.files = os.listdir(folder)
        self.hotdog_prefixes = ['chili-dog', 'frankfurter', 'hotdog']

    def __len__(self):
        return len(self.files)

    def __getitem__(self, index):
        img_name = self.files[index]
        img_path = os.path.join(self.folder, img_name)
        image = Image.open(img_path)
        y = 0
        for prefix in self.hotdog_prefixes:
            if img_name.startswith(prefix):
                y = 1
                break
        if self.transform:
            image = self.transform(image)
        return image, y, img_name
        
# Путь к обучающим данным
train_path = '/kaggle/input/hotdog-dataset-zip/Задание 5. Нейронные сети/train/train_kaggle'
test_path = '/kaggle/input/hotdog-dataset-zip/Задание 5. Нейронные сети/test/test_kaggle'

train_dataset = HotdogRecognitionDataset(train_path,
                       transform=transforms.Compose([
                           transforms.Resize((224, 224)),
                           transforms.ToTensor(),
                           # Use mean and std for pretrained models
                           # https://pytorch.org/docs/stable/torchvision/models.html
                           transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
                       ])
                      )
test_dataset = HotdogRecognitionDataset(test_path,
                       transform=transforms.Compose([
                           transforms.Resize((224, 224)),
                           transforms.ToTensor(),
                           # Use mean and std for pretrained models
                           # https://pytorch.org/docs/stable/torchvision/models.html
                           transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
                       ])
                      )
batch_size = 64

data_size = len(train_dataset)
validation_fraction = .2


val_split = int(np.floor((validation_fraction) * data_size))
indices = list(range(data_size))
np.random.seed(42)
np.random.shuffle(indices)

val_indices, train_indices = indices[:val_split], indices[val_split:]

train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                           sampler=train_sampler)
val_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                         sampler=val_sampler)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

def compute_accuracy(model, loader):
    model.eval()  # Enter evaluation mode
    correct_samples = 0
    total_samples = 0

    with torch.no_grad():
        for x, y, _ in loader:
            x = x.to(device)
            y = y.to(device)

            predictions = model(x)
            correct_samples += (predictions.argmax(dim=1) == y).sum().item()
            total_samples += y.size(0)

    return correct_samples / total_samples

def train_model_with_early_stopping(model, train_loader, val_loader, loss, optimizer, num_epochs,scheduler=None, patience=5):
    best_val_acc = 0
    patience_counter = 0
    best_model_state = None
    
    loss_history = []
    train_history = []
    val_history = []
    
    for epoch in range(num_epochs):
        model.train()
        loss_accum = 0
        correct_samples = 0
        total_samples = 0
        
        for i_step, (x, y, _) in enumerate(train_loader):
            x_gpu = x.to(device)
            y_gpu = y.to(device)
            
            prediction = model(x_gpu)
            loss_value = loss(prediction, y_gpu)
            
            optimizer.zero_grad()
            loss_value.backward()
            optimizer.step()
            
            _, indices = torch.max(prediction, 1)
            correct_samples += torch.sum(indices == y_gpu).item()
            total_samples += y.shape[0]
            loss_accum += loss_value.item()
        
        if scheduler is not None:
            scheduler.step()
        
        ave_loss = loss_accum / len(train_loader)
        train_accuracy = correct_samples / total_samples
        val_accuracy = compute_accuracy(model, val_loader)
        
        loss_history.append(ave_loss)
        train_history.append(train_accuracy)
        val_history.append(val_accuracy)
        
        print(f"Epoch {epoch+1}: Loss: {ave_loss:.4f}, Train Acc: {train_accuracy:.4f}, Val Acc: {val_accuracy:.4f}")
        
        # Ранняя остановка
        if val_accuracy > best_val_acc:
            best_val_acc = val_accuracy
            best_model_state = model.state_dict().copy()
            patience_counter = 0
        else:
            patience_counter += 1
            
        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break
            
    model.load_state_dict(best_model_state)
    return loss_history, train_history, val_history, best_val_acc
    
#Тренируем только последний слой
# for param in model.parameters():
#     param.requires_grad = False
# model.fc = nn.Linear(512, 2)
# model.fc = model.fc.to(device)
# loss = nn.CrossEntropyLoss()
# optimizer = optim.SGD( model.parameters(), lr=0.001, momentum=0.9)
# loss_history, train_history, val_history = train_model(model, train_loader, val_loader, loss, optimizer, 2)

class SubsetSampler(Sampler):

    def __init__(self, indices):
        self.indices = indices

    def __iter__(self):
        return (self.indices[i] for i in range(len(self.indices)))

    def __len__(self):
        return len(self.indices)
        
def evaluate_model(model, dataset, indices):
    sampler = SubsetSampler(indices)
    loader = DataLoader(dataset, batch_size=32, sampler=sampler)
    all_predictions = []
    all_ground_truth = []
    with torch.no_grad():
        for batch in loader:
            x, y, _ = batch
            x = x.to(device)
            y = y.to(device)
            outputs = model(x)
            _, predicted = torch.max(outputs, 1)
            all_predictions.extend(predicted.cpu().numpy())
            all_ground_truth.extend(y.cpu().numpy())
            
    predictions = np.array(all_predictions, dtype=bool)
    ground_truth = np.array(all_ground_truth, dtype=bool)
    return predictions, ground_truth

def binary_classification_metrics(prediction, ground_truth):
    prediction_int = prediction.astype(int)
    ground_truth_int = ground_truth.astype(int)
    precision = metrics.precision_score(ground_truth_int, prediction_int)
    recall = metrics.recall_score(ground_truth_int, prediction_int)
    f1 = metrics.f1_score(ground_truth_int, prediction_int)
    return precision, recall, f1

space = [
    Real(1e-5, 1e-2, name='lr', prior='log-uniform'),
    Real(1e-6, 1e-3, name='weight_decay', prior='log-uniform'),
    Categorical(['AdamW', 'SGD'], name='optimizer'),
    Categorical(['cosine', 'step', 'none'], name='scheduler'),
    Integer(10, 20, name='epochs')]

best_score = 0
best_hyperparams = None

def create_model(hyperparams):
    model = models.resnet18(pretrained=True)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 2)
    return model
    
@use_named_args(space)
def objective(**params):
    """
    Целевая функция для оптимизации
    Возвращает отрицательное значение F1-score (т.к. мы минимизируем)
    """
    global best_score, best_hyperparams
    print(f"\n=== Trial with params: {params} ===")
    model = create_model(params)
    model = model.to(device)
    
    if params['optimizer'] == 'AdamW':
        optimizer = optim.AdamW(
        model.parameters(), 
        lr=params['lr'],
        weight_decay=params['weight_decay'])
    else:
        optimizer = optim.SGD(
        model.parameters(),
        lr=params['lr'],
        weight_decay=params['weight_decay'],
        momentum=0.9)

    scheduler = None
    if params['scheduler'] == 'cosine':
        scheduler = optim.lr_scheduler.CosineAnnealingLR(
        optimizer, T_max=params['epochs'])
    elif params['scheduler'] == 'step':
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
        
    loss_fn = nn.CrossEntropyLoss()
    _, _, _, val_accuracy = train_model_with_early_stopping(
    model, train_loader, val_loader, loss_fn, optimizer, 
    params['epochs'], scheduler, patience=5)
    
    predictions, ground_truth = evaluate_model(model, train_dataset, val_indices)
    _, _, f1_score = binary_classification_metrics(predictions, ground_truth)
    print(f"Validation F1-score: {f1_score:.4f}")
    
    del model
    torch.cuda.empty_cache()
        # Возвращаем отрицательное значение F1 (минимизируем отрицательный F1 = максимизируем F1)
    return -f1_score
    
result = dummy_minimize(
        func=objective,
        dimensions=space,
        n_calls=20,
        random_state=42)
print(f"Best F1-score: {best_score:.4f}")
print(f"Best hyperparameters: {best_hyperparams}")
# predictions, ground_truth = evaluate_model(model, train_dataset, val_indices)
# precision, recall, f1 = binary_classification_metrics(predictions, ground_truth)
# print("F1: %4.3f, P: %4.3f, R: %4.3f" % (precision, recall, f1))

Using device: cuda:0

=== Trial with params: {'lr': 0.002452612631133679, 'weight_decay': 3.550621427070766e-06, 'optimizer': 'SGD', 'scheduler': 'step', 'epochs': 19} ===


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 136MB/s]


Epoch 1: Loss: 0.2639, Train Acc: 0.8765, Val Acc: 0.9587
Epoch 2: Loss: 0.0890, Train Acc: 0.9693, Val Acc: 0.9554
Epoch 3: Loss: 0.0363, Train Acc: 0.9924, Val Acc: 0.9641
Epoch 4: Loss: 0.0181, Train Acc: 0.9973, Val Acc: 0.9630
Epoch 5: Loss: 0.0093, Train Acc: 0.9992, Val Acc: 0.9630
Epoch 6: Loss: 0.0051, Train Acc: 0.9997, Val Acc: 0.9620
Epoch 7: Loss: 0.0056, Train Acc: 0.9997, Val Acc: 0.9630
Epoch 8: Loss: 0.0057, Train Acc: 0.9997, Val Acc: 0.9630
Early stopping at epoch 8
Validation F1-score: 0.9370

=== Trial with params: {'lr': 2.9375384576328278e-05, 'weight_decay': 1.493656855461764e-06, 'optimizer': 'SGD', 'scheduler': 'step', 'epochs': 17} ===


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1: Loss: 0.6900, Train Acc: 0.5854, Val Acc: 0.6728
Epoch 2: Loss: 0.5468, Train Acc: 0.7160, Val Acc: 0.7457
Epoch 3: Loss: 0.4659, Train Acc: 0.7768, Val Acc: 0.8163
Epoch 4: Loss: 0.4104, Train Acc: 0.8238, Val Acc: 0.8554
Epoch 5: Loss: 0.3756, Train Acc: 0.8482, Val Acc: 0.8674
Epoch 6: Loss: 0.3581, Train Acc: 0.8553, Val Acc: 0.8663
Epoch 7: Loss: 0.3549, Train Acc: 0.8564, Val Acc: 0.8717
Epoch 8: Loss: 0.3540, Train Acc: 0.8615, Val Acc: 0.8728
Epoch 9: Loss: 0.3514, Train Acc: 0.8569, Val Acc: 0.8772
Epoch 10: Loss: 0.3464, Train Acc: 0.8640, Val Acc: 0.8804
Epoch 11: Loss: 0.3478, Train Acc: 0.8594, Val Acc: 0.8815
Epoch 12: Loss: 0.3462, Train Acc: 0.8623, Val Acc: 0.8804
Epoch 13: Loss: 0.3464, Train Acc: 0.8615, Val Acc: 0.8761
Epoch 14: Loss: 0.3460, Train Acc: 0.8607, Val Acc: 0.8772
Epoch 15: Loss: 0.3469, Train Acc: 0.8615, Val Acc: 0.8804
Epoch 16: Loss: 0.3451, Train Acc: 0.8637, Val Acc: 0.8728
Early stopping at epoch 16
Validation F1-score: 0.7737

=== Trial

...

Epoch 1: Loss: 0.6410, Train Acc: 0.6277, Val Acc: 0.7239
Epoch 2: Loss: 0.5525, Train Acc: 0.7108, Val Acc: 0.7728
Epoch 3: Loss: 0.4953, Train Acc: 0.7507, Val Acc: 0.8087
Epoch 4: Loss: 0.4476, Train Acc: 0.7945, Val Acc: 0.8370
Epoch 5: Loss: 0.4151, Train Acc: 0.8200, Val Acc: 0.8500
Epoch 6: Loss: 0.3993, Train Acc: 0.8338, Val Acc: 0.8478
Epoch 7: Loss: 0.3962, Train Acc: 0.8322, Val Acc: 0.8554
Epoch 8: Loss: 0.3942, Train Acc: 0.8390, Val Acc: 0.8489
Epoch 9: Loss: 0.3901, Train Acc: 0.8376, Val Acc: 0.8522
Epoch 10: Loss: 0.3880, Train Acc: 0.8436, Val Acc: 0.8543
Epoch 11: Loss: 0.3877, Train Acc: 0.8395, Val Acc: 0.8587
Epoch 12: Loss: 0.3891, Train Acc: 0.8368, Val Acc: 0.8554
Epoch 13: Loss: 0.3872, Train Acc: 0.8401, Val Acc: 0.8620
Epoch 14: Loss: 0.3876, Train Acc: 0.8368, Val Acc: 0.8565
Epoch 15: Loss: 0.3845, Train Acc: 0.8428, Val Acc: 0.8620
Epoch 16: Loss: 0.3851, Train Acc: 0.8390, Val Acc: 0.8609
Epoch 17: Loss: 0.3862, Train Acc: 0.8376, Val Acc: 0.8489
Valida

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1: Loss: 0.2307, Train Acc: 0.9014, Val Acc: 0.9587
Epoch 2: Loss: 0.0395, Train Acc: 0.9878, Val Acc: 0.9620
Epoch 3: Loss: 0.0123, Train Acc: 0.9986, Val Acc: 0.9598
Epoch 4: Loss: 0.0053, Train Acc: 0.9997, Val Acc: 0.9652
Epoch 5: Loss: 0.0021, Train Acc: 1.0000, Val Acc: 0.9663
Epoch 6: Loss: 0.0032, Train Acc: 1.0000, Val Acc: 0.9663
Epoch 7: Loss: 0.0022, Train Acc: 0.9997, Val Acc: 0.9641
Epoch 8: Loss: 0.0023, Train Acc: 0.9997, Val Acc: 0.9674
Epoch 9: Loss: 0.0019, Train Acc: 1.0000, Val Acc: 0.9652
Epoch 10: Loss: 0.0006, Train Acc: 1.0000, Val Acc: 0.9630
Epoch 11: Loss: 0.0008, Train Acc: 1.0000, Val Acc: 0.9630
Epoch 12: Loss: 0.0006, Train Acc: 1.0000, Val Acc: 0.9630
Validation F1-score: 0.9361

=== Trial with params: {'lr': 0.0019158219548093176, 'weight_decay': 4.8287152161792125e-05, 'optimizer': 'SGD', 'scheduler': 'step', 'epochs': 18} ===


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1: Loss: 0.2566, Train Acc: 0.8789, Val Acc: 0.9620
Epoch 2: Loss: 0.0972, Train Acc: 0.9628, Val Acc: 0.9620
Epoch 3: Loss: 0.0498, Train Acc: 0.9875, Val Acc: 0.9630
Epoch 4: Loss: 0.0257, Train Acc: 0.9951, Val Acc: 0.9609
Epoch 5: Loss: 0.0160, Train Acc: 0.9981, Val Acc: 0.9630
Epoch 6: Loss: 0.0108, Train Acc: 1.0000, Val Acc: 0.9630
Epoch 7: Loss: 0.0108, Train Acc: 0.9981, Val Acc: 0.9630
Epoch 8: Loss: 0.0100, Train Acc: 0.9992, Val Acc: 0.9630
Early stopping at epoch 8
Validation F1-score: 0.9368
Best F1-score: 0.0000
Best hyperparameters: None
